In [ ]:
# Code to Extract the tweet times from elonmusk.csv 

import csv
import re

def clean_line(line): # Get the last two lines while cleaning the weird characters
    new_line = re.sub(r'[^\w\s,:"]', '', line) 
    if len(new_line.split(',')) < 2: 
        return ' '
    second_last_entry = new_line.split(',')[-2] 
    last_entry = new_line.split(',')[-1]
    return second_last_entry + last_entry  

def starts_and_ends_with_quote(s):
    return s.strip().startswith('"') and s.strip().endswith('"')

# Read the CSV file with the specified encoding
with open('elonmusk.csv', 'r', encoding='utf-8', errors='ignore') as file: # Filters the bad characters
    lines = file.readlines()

cleaned_lines = [clean_line(line) for line in lines[1:]] 
filtered_lines = [line.lstrip() for line in cleaned_lines if starts_and_ends_with_quote(line)] # Filters only the time-stamps
final_lines = [line for line in filtered_lines if len(line) < 26] # Removes all the non-time stamps

# Write the modified lines back to the CSV file
with open('fitlered_times.csv', 'w', encoding='utf-8') as file:
    file.writelines(final_lines)



In [62]:
# Code to Extract the IDS from elonmusk.csv

import csv
import re

def clean_line(line):
    # Remove ambiguous characters but keep spaces, newlines, and commas
    new_line = re.sub(r'[^\w\s,:"]', '', line)
    
    # Split the cleaned line by commas and get the last entry
    first_entry = new_line.split(',')[0]
    return first_entry + '\n'

def has_19_digits(s):
    return bool(re.match(r'^\d{19}$', s.strip()))

# Read the CSV file with the specified encoding
with open('elonmusk.csv', 'r', encoding='utf-8', errors='ignore') as file:
    lines = file.readlines()

# Clean each line
cleaned_lines = [clean_line(line) for line in lines[1:]]
filtered_lines = [line for line in cleaned_lines if has_19_digits(line)]

# Write the modified lines back to the CSV file
with open('ids.csv', 'w', encoding='utf-8') as file:
    file.writelines(filtered_lines)

In [83]:
# Code to Extract whether the tweet is a reply or not from elonmusk.csv

def clean_line(line):
    # Remove ambiguous characters but keep spaces, newlines, and commas
    new_line = re.sub(r'[^\w\s,:"@]', '', line)
    if len(new_line.split(',')) < 2:
        return ''
    first_entry = new_line.split(',')[0]
    second_entry = new_line.split(',')[1]
    output_string = first_entry + ',' + second_entry
    
    if len(output_string) > 22:
        s = output_string[21:]
        if bool(re.match(r'^RT', s.strip())):
            if output_string[-1] == '\n':
                return output_string
            return output_string + '\n'
    return ''
    

# Read the CSV file with the specified encoding
with open('elonmusk.csv', 'r', encoding='utf-8', errors='ignore') as file:
    lines = file.readlines()

# Clean each line
cleaned_lines = [clean_line(line) for line in lines[1:]]
modified_lines = [line.rstrip('\n') + '"' + '\n' if not line.rstrip('\n').endswith('"') else line for line in cleaned_lines]
modified_lines = [line for line in modified_lines if not len(line) == 2]

# Write the modified lines back to the CSV file
with open('retweets.csv', 'w', encoding='utf-8') as file:
    file.writelines(modified_lines)


In [84]:
import pandas as pd

In [85]:
def get_retweeted_from(s):
    first_entry = s.split(':')[0]
    return first_entry[4:]

df = pd.read_csv('retweets.csv', header=None)
df.columns = ['ID', 'Tweet']
df['Retweeted'] = df['Tweet'].apply(get_retweeted_from)

df.head(20)
print(df['Tweet'][1])

RT @MarioNawfal:  BRAZILIAN X NEWS OUTLET LATEST CENSORSHIP VICTIM


In [86]:
ids = pd.read_csv('ids.csv', header=None)
ids.columns = ['ID']

In [ ]:
# Read the ids.csv file
ids = pd.read_csv('ids.csv', header=None)
ids.columns = ['ID']

# Read the retweets.csv file
df = pd.read_csv('retweets.csv', header=None)
df.columns = ['ID', 'Tweet']
df['Retweeted'] = df['Tweet'].apply(get_retweeted_from)

# Merge the two DataFrames on the 'ID' column, keeping all IDs from ids
merged_df = pd.merge(ids, df, on='ID', how='left')

# Write the merged DataFrame to a new CSV file
merged_df.head(20)

# Write the merged DataFrame to a new CSV file
merged_df.to_csv('final.csv', index=False)

KeyError: 'Tweet'